In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import re

In [2]:
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'

In [3]:
r = requests.get(url)
json = r.json()

In [5]:
players = pd.DataFrame(json['elements'])
players[['id', 'web_name', 'team', 'element_type','total_points','now_cost']].head()

,id,web_name,team,element_type,total_points,now_cost
0,1,Cédric,1,2,48,45
1,2,Leno,1,1,10,45
2,3,Xhaka,1,3,60,50
3,4,Elneny,1,3,27,45
4,5,Holding,1,2,29,45


In [6]:
position = np.array([json['element_types'][i]['singular_name_short'] for i in range(4)])

In [7]:
position

array(['GKP', 'DEF', 'MID', 'FWD'], dtype='<U3')

In [8]:
pos = players['element_type'].values

In [9]:
pos_word = position[pos-1]

In [10]:
players['element_type'] = pos_word

In [11]:
players[['id', 'web_name', 'team', 'element_type','total_points','now_cost']].head()

,id,web_name,team,element_type,total_points,now_cost
0,1,Cédric,1,DEF,48,45
1,2,Leno,1,GKP,10,45
2,3,Xhaka,1,MID,60,50
3,4,Elneny,1,MID,27,45
4,5,Holding,1,DEF,29,45


In [12]:
players['ppc'] = players['total_points']/players['now_cost']

In [13]:
players = players.sort_values('ppc', ascending=False)

In [14]:
players = players[['web_name', 'team', 'element_type','total_points','now_cost','ppc']]

In [15]:
teams = np.array([json['teams'][i]['name'] for i in range(len(json['teams']))])

In [16]:
team_ids = players['team'].values

In [17]:
team_word = teams[team_ids-1]

In [18]:
players['team'] = team_word

In [19]:
GKPs = players[players['element_type']=='GKP']
DEFs = players[players['element_type']=='DEF']
MIDs = players[players['element_type']=='MID']
FWDs = players[players['element_type']=='FWD']

In [21]:
writer = pd.ExcelWriter('FPL_Data.xlsx')
GKPs.to_excel(writer,'Goalkeepers')
DEFs.to_excel(writer,'Defenders')
MIDs.to_excel(writer,'Midfielders')
FWDs.to_excel(writer,'Forwards')
writer.save()